In [18]:
import pandas as pd
import numpy

#learning stuff
from sklearn import neighbors
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

#graphing stuff!
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [15]:
data = pd.read_excel('DataRefined.xlsx')
#note, data is a pandas DataFrame
data_array = data.values
#data_array is a bit easier to access - each value can be accessed with data_array[x]

In [16]:
print(data.head())

   track_id  acousticness  danceability    energy  instrumentalness  liveness  \
0     19469      0.078285      0.314572  0.309468          0.089045  0.354460   
1     26170      0.907184      0.608202  0.487272          0.952051  0.147523   
2      3628      0.902820      0.417216  0.192552          0.941335  0.103392   
3      4005      0.514903      0.297365  0.716660          0.884307  0.164822   
4     11891      0.420525      0.419414  0.800006          0.917086  0.188013   

   speechiness    tempo   valence      artist_name  \
0     0.051375  127.856  0.212025       Deerhunter   
1     0.055712  127.945  0.655503      Monk Turner   
2     0.034817  110.647  0.114613      Fern Knight   
3     0.045856  157.893  0.704008  MORS ONTOLOGICA   
4     0.046945   98.782  0.856437       Wildabeast   

                                           track_url  happysad  calmenergetic  \
0  http://freemusicarchive.org/music/Barry_Chabal...       NaN            NaN   
1  http://freemusicarchive

In [17]:
print(data_array[0])

[19469 0.0782847905 0.3145720836 0.3094678407 0.0890453704 0.3544597401
 0.0513748433 127.856 0.2120248415 'Deerhunter'
 'http://freemusicarchive.org/music/Barry_Chabala/Feedback/new_improved_lowfat_eric'
 nan nan nan 0.02604127702184378]
